# M3E2

## Author: Raquel Aoki

Date: Spring 2021


In [1]:
!git clone https://github.com/raquelaoki/ParKCa.git
!git clone https://github.com/raquelaoki/CompBioAndSimulated_Datasets.git
#!git clone https://github.com/JakeColtman/bartpy.git

fatal: destination path 'ParKCa' already exists and is not an empty directory.
fatal: destination path 'CompBioAndSimulated_Datasets' already exists and is not an empty directory.


In [2]:
import random
import pandas as pd
import numpy as np
import sys
import yaml
import time
from sklearn.model_selection import train_test_split
import torch

sys.path.insert(0, 'src/')
#sys.path.insert(0, 'bartpy/')  # https://github.com/JakeColtman/bartpy
sys.path.insert(0, 'ParKCa/src/')
#from ParKCa.src.train import *
from CompBioAndSimulated_Datasets.simulated_data_multicause import *
import model_m3e2 as m3e2


def main(config_path, seed_models, seed_data):
    """Start: Parameters Loading"""
    with open(config_path) as f:
        config = yaml.safe_load(f)
    params = config['parameters']

    # Fix numpy seed for reproducibility
    np.random.seed(seed_models)
    # Fix random seed for reproducibility
    random.seed(seed_models)
    # Fix Torch graph-level seed for reproducibility
    torch.manual_seed(seed_models)

    if 'gwas' in params['data']:

        params_b = {'DA': {'k': [15]},
                    'CEVAE': {'num_epochs': 100, 'batch': 200, 'z_dim': 10}}

        params["n_treatments"] = trykey(params, 'n_treatments', 5)
        prop = params["n_treatments"] / (params["n_treatments"] + params['n_covariates'])

        sdata_gwas = gwas_simulated_data(prop_tc=prop,
                                         pca_path='CompBioAndSimulated_Datasets/data/tgp_pca2.txt',
                                         seed=seed_data,
                                         n_units=params['n_sample'],
                                         n_causes=params["n_treatments"] + params['n_covariates'],
                                         true_causes=params["n_treatments"])
        X, y, y01, treatement_columns, treatment_effects, group = sdata_gwas.generate_samples()
        # Train and Test split use the same seed
        params['baselines'] = trykey(params, 'baselines', False)
        if params['baselines']:
            baselines_results, exp_time, f1_test = baselines(params['baselines_list'], pd.DataFrame(X), y01, params_b,
                                                             TreatCols=treatement_columns, timeit=True,
                                                             seed=seed_models)
        else:
            print('Im constructing the noise')
            baselines_results, exp_time, f1_test = baselines(['noise'], pd.DataFrame(X), y01, params_b,
                                                             TreatCols=treatement_columns, timeit=True,
                                                             seed=seed_models)

        start_time = time.time()
        X_train, X_test, y_train, y_test = train_test_split(X, y01, test_size=0.33, random_state=seed_models)
        print('... Target - proportion of 1s', np.sum(y01) / len(y01))
        # Split X1, X2 on GWAS: case with no clinicla variables , X2 = X
        X1_cols = []
        X2_cols = range(X.shape[1] - len(treatement_columns))

        data_nnl = m3e2.data_nn(X_train.values, X_test.values, y_train, y_test, treatement_columns,
                                treatment_effects[treatement_columns], X1_cols, X2_cols)
        loader_train, loader_val, loader_test, num_features = data_nnl.loader(params['suffle'], params['batch_size'],
                                                                              seed_models)
        params['pos_weights'] = data_nnl.treat_weights
        params['pos_weight_y'] = trykey(params, 'pos_weight_y', 1)
        params['hidden1'] = trykey(params, 'hidden1', 64)
        params['hidden2'] = trykey(params, 'hidden2', 8)
        cate_m3e2, f1_test_ = m3e2.fit_nn(loader_train, loader_val, loader_test, params, treatement_columns,
                                          num_features,
                                          X1_cols, X2_cols)
        print('... CATE')
        baselines_results['M3E2'] = cate_m3e2
        exp_time['M3E2'] = time.time() - start_time
        f1_test['M3E2'] = f1_test_
        output = organize_output(baselines_results.copy(), treatment_effects[treatement_columns], exp_time, f1_test)
    if 'copula' in params['data']:
        params_b = {'DA': {'k': [5]},
                    'CEVAE': {'num_epochs': 100, 'batch': 200, 'z_dim': 5}}

        sdata_copula = copula_simulated_data(seed=seed_data, n=params['n_sample'], s=params['n_covariates'])
        X, y, y01, treatement_columns, treatment_effects = sdata_copula.generate_samples()

        if params['baselines']:
            baselines_results, exp_time, f1_test = baselines(params['baselines_list'], pd.DataFrame(X), y01, params_b,
                                                             TreatCols=treatement_columns, timeit=True,
                                                             seed=seed_models)
        else:
            baselines_results, exp_time, f1_test = baselines(['noise'], pd.DataFrame(X), y01, params_b,
                                                             TreatCols=treatement_columns, timeit=True,
                                                             seed=seed_models)
        start = time.time()
        X_train, X_test, y_train, y_test = train_test_split(X, y01, test_size=0.33, random_state=seed_models)
        X1_cols = []
        X2_cols = range(X.shape[1] - len(treatement_columns))
        # TODO: add other baselines here to run everything on the same train/testing sets

        data_nnl = m3e2.data_nn(X_train, X_test, y_train, y_test, treatement_columns,
                                treatment_effects, X1_cols, X2_cols)
        loader_train, loader_val, loader_test, num_features = data_nnl.loader(params['suffle'], params['batch_size'],
                                                                              seed_models)
        params['pos_weights'] = data_nnl.treat_weights
        params['pos_weight_y'] = trykey(params, 'pos_weight_y', 1)
        params['hidden1'] = trykey(params, 'hidden1', 6)
        params['hidden2'] = trykey(params, 'hidden2', 6)

        cate_m3e2, f1_test_ = m3e2.fit_nn(loader_train, loader_val, loader_test, params, treatement_columns,
                                          num_features,
                                          X1_cols, X2_cols)
        print('... CATE')
        cate = pd.DataFrame({'CATE_M3E2': cate_m3e2, 'True_Effect': treatment_effects})
        baselines_results['M3E2'] = cate_m3e2
        exp_time['M3E2'] = time.time() - start_time
        f1_test['M3E2'] = f1_test_
        output = organize_output(baselines_results.copy(), treatment_effects[treatement_columns], exp_time, f1_test)
    if 'gwas' not in params['data'] and 'copula' not in params['data']:
        print(
            "ERRROR! \nDataset not recognized. \nChange the parameter data in your config.yaml file to gwas or copula.")

    name = 'output_' + params['data'][0] + '_' + params['id'] + '.csv'
    output['seed_data'] = seed_data
    output['seed_models'] = seed_models

    return output, name


def trykey(params, key, default):
    try:
        return params[key]
    except KeyError:
        params[key] = default
        return params[key]


def baselines(BaselinesList, X, y, ParamsList, seed=63, TreatCols=None, id='', timeit=False):
    """
    input:
        X, colnamesX: potential causes and their names
        Z, colnamesZ: confounders and their names
        y: 01 outcome
        causes: name of the potential causes
    """

    if TreatCols is None:
        TreatCols = list(range(X.shape[1]))

    # check if binary treatments
    X01 = X.copy()
    for col in TreatCols:
        a = X01.iloc[:, col]
        if not ((a == 0) | (a == 1)).all():
            mean_v = np.mean(X01.iloc[:, col])
            X01.iloc[:, col] = [1 if i > mean_v else 0 for i in X01.iloc[:, col]]
        else:
            pass

    X_train, X_test, y_train, y_test, X_train01, X_test01 = train_test_split(X, y, X01,
                                                                             test_size=0.33, random_state=seed)
    coef_table = pd.DataFrame(columns=['causes'])
    coef_table['causes'] = ['T' + str(i) for i in range(len(TreatCols))]
    times, f1_test = {}, {}

    if 'DA' in BaselinesList:
        start_time = time.time()
        from deconfounder import deconfounder_algorithm as DA
        ParamsList['DA']['k'] = trykey(ParamsList['DA'], 'k', 15)  # if exploring multiple latent sizes
        for k in ParamsList['DA']['k']:
            if len(ParamsList['DA']['k']) > 1:
                coln = 'DA_' + str(id) + str(k)
            else:
                coln = 'DA'
            model_da = DA(X_train, X_test, y_train, y_test, k, print_=False)
            ParamsList['DA']['class_weight'] = trykey(ParamsList['DA'], 'class_weight', {0: 1, 1: 1})
            coef, coef_continuos, roc, f1_test['DA'] = model_da.fit(class_weight=ParamsList['DA']['class_weight'])
            coef_table[coln] = coef_continuos[TreatCols]
        times['DA'] = time.time() - start_time
        print('\nDone!')

    if 'BART' in BaselinesList:
        start_time = time.time()
        from bart import BART as BART
        model_bart = BART(X_train01, X_test01, y_train, y_test)
        ParamsList['BART']['n_trees'] = trykey(ParamsList['BART'], 'n_trees', 50)
        ParamsList['BART']['n_burn'] = trykey(ParamsList['BART'], 'n_burn', 100)
        model_bart.fit(n_trees=ParamsList['BART']['n_trees'], n_burn=ParamsList['BART']['n_burn'], print_=False)
        print('...... predictions')
        coef_table['BART'], f1_test['BART'] = model_bart.cate(TreatCols, print_=False)
        times['BART'] = time.time() - start_time
        print('\nDone!')

    if 'CEVAE' in BaselinesList:
        print('\n\n Learner: CEVAE')
        start_time = time.time()
        from cevae import CEVAE as CEVAE
        print('Note: Treatments should be the first columns of X')
        ParamsList['CEVAE']['epochs'] = trykey(ParamsList['CEVAE'], 'epochs', 100)
        ParamsList['CEVAE']['batch'] = trykey(ParamsList['CEVAE'], 'batch', 200)
        ParamsList['CEVAE']['z_dim'] = trykey(ParamsList['CEVAE'], 'z_dim', 5)

        confeatures, binfeatures = [], []
        for col in range(X_train01.shape[1]):
            a = X_train01.iloc[:, col]
            if not ((a == 0) | (a == 1)).all():
                confeatures.append(col)
            else:
                binfeatures.append(col)

        print('... length con and bin features', len(confeatures), len(binfeatures))
        model_cevae = CEVAE(X_train01, X_test01, y_train, y_test, TreatCols,
                            binfeats=binfeatures, contfeats=confeatures,
                            epochs=ParamsList['CEVAE']['epochs'],
                            batch=ParamsList['CEVAE']['batch'],
                            z_dim=ParamsList['CEVAE']['z_dim'])
        coef_table['CEVAE'], f1_test['CEVAE'] = model_cevae.fit_all(print_=False)
        times['CEVAE'] = time.time() - start_time
        print('\nDone!')

    if not timeit:
        return coef_table
    else:
        return coef_table, times, f1_test


def organize_output(experiments, true_effect, exp_time=None, f1_scores=None):
    """
    Important: experiments, experiments times and f1 scores should be in the same order
    Parameters
    ----------
    experiments
    true_effect
    exp_time

    Returns
    -------
    """
    Treatments = experiments['causes']
    experiments.set_index('causes', inplace=True)
    experiments['TrueTreat'] = true_effect
    Treatments_cate = np.transpose(experiments)
    BaselinesNames = experiments.columns
    mae = []
    for col in BaselinesNames:
        dif = np.abs(experiments[col] - experiments['TrueTreat'])
        mae.append(np.nanmean(dif))
    output = pd.DataFrame({'Method': BaselinesNames, 'MAE': mae})
    exp_time['TrueTreat'] = 0
    f1_scores['TrueTreat'] = 0
    if f1_scores is not None:
        output['F1_Test'] = [f1_scores[m] for m in output['Method'].values]
    if exp_time is not None:
        output['Time(s)'] = [exp_time[m] for m in output['Method'].values]

    out = pd.DataFrame(Treatments_cate, columns=Treatments)
    out.reset_index(inplace=True, drop=True)

    return pd.concat((output, out), 1)


colab = False
notebook = True
arg = {'config_path': 'config1.yaml',
       'seed_models': 3,
       'seed_data': 2,
       }
if colab:
    arg['path'] = '/content/'
    arg['config_path'] = arg['path']+arg['config_path']
else:
    arg['path'] = ''

if __name__ == "__main__":
    start_time = time.time()
    if notebook:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Cuda Availble:", torch.cuda.is_available(), " device: ", device)
        for j in range(arg['seed_data']):
            print('Data',j)
            for i in range(arg['seed_models']):
                print('Models',i)
                if i == 0 and j == 0:
                    output, name = main(config_path=arg['config_path'], seed_models=i, seed_data=j)
                else:
                    output_, name = main(config_path=arg['config_path'], seed_models=i, seed_data=j)
                    output = pd.concat([output, output_], 0, ignore_index=True)
    else:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Cuda Availble:", torch.cuda.is_available(), " device: ", device)
        for j in range(sys.argv[3]):
            print('Data', j)
            for i in range(sys.argv[2]):
                print('Models', i)
                if i == 0:
                    output, name = main(config_path=sys.argv[1], seed_models=i+1, seed_data=j+1)
                else:
                    output_, name = main(config_path=sys.argv[1], seed_models=i+1, seed_data=j+1)
                    output = pd.concat([output, output_], 0, ignore_index=True)

    output.to_csv(name)
    end_time = time.time() - start_time
    end_time_m = end_time / 60
    end_time_h = end_time_m / 60
    print("Time ------ {} min / {} hours ------".format(end_time_m, end_time_h))


Cuda Availble: True  device:  cuda
Data 0
Models 0
GWAS simulated data initialized!
...  5 true causes and  995  confounders
... Treatments:  5  proportions  [0.049, 0.045, 0.065, 0.131, 0.154]
... Confounders:  995
... Target (y) : 0.378
... Sample Size: 1000
 Data Simulation Done!
... Target - proportion of 1s 0.378
M3E2: Train Shape  (670, 995) (670, 5)
... Model initialization done!
... Training
CHECKING LOSSES BEFORE WEIGHT: 6.713869 0.7777651 0.12893267
CHECKING LOSSES AFTER WEIGHT: 6.713869094848633 7.777650952339172 1.289326697587967
......  0  
... Train: loss  17.59 0.13 metric  [0.15982456 0.0777193  0.11815789 0.83245614 0.07438596 0.36885965] 
... Val: loss  13.87 metric  [0.05940594 0.08910891 0.07920792 0.87128713 0.04950495 0.4950495 ]
CHECKING LOSSES BEFORE WEIGHT: 6.485884 0.58392054 0.1129215
CHECKING LOSSES AFTER WEIGHT: 6.485884189605713 5.839205384254456 1.1292149871587753
......  15  
... Train: loss  13.99 0.12 metric  [0.15710526 0.9395614  0.54157895 0.7435087

... Loading Best validation (epoch  122 )
... Final Metrics - Target
...... Train :  0.66
...... Val :  0.564
...... Test :  0.621
CHANGED FROM F1 TO ACC
Outcome Y [ 3.4641113e-02 -1.5991354e-01  4.3313306e-02  1.9054779e-01
  2.7417162e-01  1.3314378e-01  5.2116497e-04 -1.5449357e-01
  7.4790460e-01]
... CATE
Models 2
GWAS simulated data initialized!
...  5 true causes and  995  confounders
... Treatments:  5  proportions  [0.049, 0.045, 0.065, 0.131, 0.154]
... Confounders:  995
... Target (y) : 0.378
... Sample Size: 1000
 Data Simulation Done!
... Target - proportion of 1s 0.378
M3E2: Train Shape  (670, 995) (670, 5)
... Model initialization done!
... Training
CHECKING LOSSES BEFORE WEIGHT: 8.22994 0.9375097 0.10960824
CHECKING LOSSES AFTER WEIGHT: 8.229940414428711 9.375097155570984 1.096082404255867
......  0  
... Train: loss  17.88 0.13 metric  [0.15754386 0.10754386 0.11982456 0.86017544 0.07815789 0.35798246] 
... Val: loss  14.69 metric  [0.13861386 0.05940594 0.12871287 0.8

CHECKING LOSSES BEFORE WEIGHT: 3.1790233 0.118813775 0.11616491
CHECKING LOSSES AFTER WEIGHT: 3.179023265838623 1.1881377547979355 1.161649078130722
......  135  
... Train: loss  6.1 0.12 metric  [0.99       0.88017544 0.64429825 0.92833333 0.8645614  0.98833333] 
... Val: loss  25.1 metric  [0.91089109 0.62376238 0.59405941 0.74257426 0.69306931 0.52475248]
... Loading Best validation (epoch  59 )
... Final Metrics - Target
...... Train :  0.94
...... Val :  0.535
...... Test :  0.497
CHANGED FROM F1 TO ACC
Outcome Y [ 0.21573746  0.09309998 -0.15594824 -0.25101283  0.4664622   0.25512287
  1.19721     0.10369409  0.9071448 ]
... CATE
Models 1
GWAS simulated data initialized!
...  5 true causes and  995  confounders
... Treatments:  5  proportions  [0.149, 0.14, 0.161, 0.126, 0.049]
... Confounders:  995
... Target (y) : 0.464
... Sample Size: 1000
 Data Simulation Done!
... Target - proportion of 1s 0.464
M3E2: Train Shape  (670, 995) (670, 5)
... Model initialization done!
... Trai

CHECKING LOSSES BEFORE WEIGHT: 6.6637325 0.035056446 0.08938764
CHECKING LOSSES AFTER WEIGHT: 6.663732528686523 0.35056445747613907 0.8938764035701752
......  135  
... Train: loss  8.89 0.12 metric  [0.48070175 0.76140351 0.49236842 0.58078947 0.11649123 0.98833333] 
... Val: loss  23.99 metric  [0.55445545 0.65346535 0.61386139 0.38613861 0.0990099  0.55445545]
... Loading Best validation (epoch  83 )
... Final Metrics - Target
...... Train :  0.96
...... Val :  0.584
...... Test :  0.53
CHANGED FROM F1 TO ACC
Outcome Y [ 0.2785208   0.31617874  0.2949473   0.02836031 -0.28197017  0.2616703
 -1.0404075   1.0193583   0.01325624]
... CATE
Time ------ 5.544817837079366 min / 0.09241363061798943 hours ------


In [3]:
#CEVAE IS KILLING IT

In [4]:
output 

,Method,MAE,F1_Test,Time(s),T0,T1,T2,T3,T4,seed_data,seed_models
0,M3E2,0.294364,0.475758,47.598127,0.468033,0.386212,0.005711,-0.091287,0.735034,0,0
1,TrueTreat,0.000000,0.000000,0.000000,0.441013,0.100039,0.244684,0.560223,0.466889,0,0
2,M3E2,0.286018,0.621212,85.093912,0.034641,-0.159914,0.043313,0.190548,0.274172,0,1
3,TrueTreat,0.000000,0.000000,0.000000,0.441013,0.100039,0.244684,0.560223,0.466889,0,1
4,M3E2,0.434867,0.600000,40.138783,0.201929,0.148854,-0.034404,-0.290477,-0.289757,0,2
5,TrueTreat,0.000000,0.000000,0.000000,0.441013,0.100039,0.244684,0.560223,0.466889,0,2
6,M3E2,0.145527,0.496970,42.187582,0.215737,0.093100,-0.155948,-0.251013,0.466462,1,0
7,TrueTreat,0.000000,0.000000,0.000000,0.406086,-0.152939,-0.132043,-0.268242,0.216352,1,0
8,M3E2,0.405768,0.551515,65.079759,-0.020600,0.205032,0.195881,0.491587,0.059924,1,1
9,TrueTreat,0.000000,0.000000,0.000000,0.406086,-0.152939,-0.132043,-0.268242,0.216352,1,1


In [5]:
output.groupby(['seed_data','Method']).mean()

MAE   F1_Test    Time(s)        T0        T1  \
seed_data Method                                                         
0         M3E2       0.338416  0.565657  57.610274  0.234868  0.125051   
          TrueTreat  0.000000  0.000000   0.000000  0.441013  0.100039   
1         M3E2       0.305005  0.526263  49.315907  0.157886  0.204770   
          TrueTreat  0.000000  0.000000   0.000000  0.406086 -0.152939   

                           T2        T3        T4  seed_models  
seed_data Method                                                
0         M3E2       0.004873 -0.063739  0.239816            1  
          TrueTreat  0.244684  0.560223  0.466889            1  
1         M3E2       0.111627  0.089645  0.081472            1  
          TrueTreat -0.132043 -0.268242  0.216352            1

In [6]:
import torch
a = torch.randn(2, 2)
print('Available devices ', torch.cuda.device_count())
print('Current cuda device ', torch.cuda.current_device())
cuda = torch.device(0)
b = a.cuda()
print(a)
print(b)

Available devices  1
Current cuda device  0
tensor([[-0.4045, -0.6741],
        [-1.4489,  0.8473]])
tensor([[-0.4045, -0.6741],
        [-1.4489,  0.8473]], device='cuda:0')
